In [19]:
import requests
import json, ndjson
from collections import Counter

with open("secret.csv", "r") as r:
    data = [line.replace("\n","").split(",") for line in r.readlines()]
    username = data[0][1]
    token = data[1][1]

In [2]:
url = "https://lichess.org"
print(f"Gathering data for {username}")
response = requests.get(
  f'https://lichess.org/api/games/user/{username}',
  params={
    "perfType" : "blitz",
    "rated" :  "true",
    "opening" : "true",
    "moves" : "false"
  },
  headers={
    'Authorization': f'Bearer {token}', # Need this or you will get a 401: Not Authorized response
    "Accept": "application/x-ndjson"
  }
)

Gathering data for sirsnakerb


In [3]:
# Parse application/x-ndjson into list of JSON objects
games = []
ndjson = response.content.decode().split('\n')

for json_obj in ndjson:
    if json_obj:
        games.append(json.loads(json_obj))

In [4]:
len(games)

3685

In [5]:
games[1]

{'id': '8CUOWCNu',
 'rated': True,
 'variant': 'standard',
 'speed': 'blitz',
 'perf': 'blitz',
 'createdAt': 1603039771257,
 'lastMoveAt': 1603040105442,
 'status': 'mate',
 'players': {'white': {'user': {'name': 'sirsnakerb', 'id': 'sirsnakerb'},
   'rating': 1412,
   'ratingDiff': 5},
  'black': {'user': {'name': 'ITHM', 'id': 'ithm'},
   'rating': 1338,
   'ratingDiff': -4}},
 'winner': 'white',
 'opening': {'eco': 'C00',
  'name': 'French Defense: La Bourdonnais Variation',
  'ply': 3},
 'clock': {'initial': 300, 'increment': 0, 'totalTime': 300}}

In [15]:
# seperate games for black and white games
white_games, black_games = [], []
for game in games:
    if game.get("opening"):
        if game.get("players").get("white").get("user").get("name") == username:
            white_games.append(game)
        else:
            black_games.append(game)

In [31]:
top = 5
for colour, col_games in zip(["white", "black"],[white_games, black_games]):
    wins, lost, draws = [], [], []
    for game in col_games:
        if game.get("winner") == colour:
            wins.append(game)
        elif game.get("winner") == None:
            draws.append(game)
        else:
            lost.append(game)
    print(f"Postions when playing {colour}")
    
    for conc, str_conc in zip([wins, draws, tie],["winning", "lost", " playing draw"]):
        print(f"Postions when {str_conc}:")
        most_common = Counter([game["opening"]["name"] for game in conc]).most_common()
        most_common_percentage = [(opening[0], round(opening[1]/len(conc)*100,2)) for opening in most_common]
        for i, opn in enumerate(most_common_percentage):
            if i == top:
                break
            print(f"{opn[0]:<47}| {opn[1]:<5} %")


Postions when playing white
Postions when winning:
Scotch Game                                    | 7.03  %
Amazon Attack                                  | 6.79  %
Queen's Pawn Game                              | 6.18  %
Philidor Defense                               | 4.12  %
Sicilian Defense: McDonnell Attack             | 3.15  %
Postions when lost:
Scotch Game                                    | 12.12 %
Amazon Attack                                  | 7.07  %
Philidor Defense                               | 5.05  %
Scandinavian Defense: Mieses-Kotroc Variation  | 4.04  %
Queen's Pawn Game                              | 4.04  %
Postions when  playing draw:
Amazon Attack                                  | 7.49  %
Scotch Game                                    | 5.69  %
Queen's Pawn Game                              | 4.39  %
Horwitz Defense                                | 2.79  %
Philidor Defense                               | 2.69  %
Postions when playing black
Postions when win